In [1]:
from dataclasses import dataclass
import re

In [2]:
@dataclass
class Vector:
    x: int = 0
    y: int = 0
    z: int = 0

@dataclass
class Moon:
    pos: Vector
    vel: Vector = Vector()
        
    def energy(self):
        pass

In [3]:
moons = []
with open("day12-test1.input") as file:
    for line in file:
        moon = Moon(pos=Vector(*map(int, re.findall(r"=(-?\d+)", line))))
        moons.append(moon)

In [4]:
moons

[Moon(pos=Vector(x=-1, y=0, z=2), vel=Vector(x=0, y=0, z=0)),
 Moon(pos=Vector(x=2, y=-10, z=-7), vel=Vector(x=0, y=0, z=0)),
 Moon(pos=Vector(x=4, y=-8, z=8), vel=Vector(x=0, y=0, z=0)),
 Moon(pos=Vector(x=3, y=5, z=-1), vel=Vector(x=0, y=0, z=0))]